<a href="https://colab.research.google.com/github/Chehmet/Triple_mnist/blob/main/CNN___NIC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install datasets

In [27]:
import pandas
from datasets import load_dataset
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import re
import tensorflow as tf
from tensorflow import keras
import zipfile
import random
import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [8]:
!git clone https://huggingface.co/datasets/khushpatel2002/triple-mnist


fatal: destination path 'triple-mnist' already exists and is not an empty directory.


In [19]:
trainpath = '/content/triple-mnist/train'
testpath = '/content/triple-mnist/test'
valpath = '/content/triple-mnist/val'
train_dir = [name for name in os.listdir(trainpath) if os.path.isdir(os.path.join(trainpath, name))]
test_dir  = [name for name in os.listdir(testpath) if os.path.isdir(os.path.join(testpath, name))]
val_dir =  [name for name in os.listdir(valpath) if os.path.isdir(os.path.join(valpath, name))]
train_dir.sort()
test_dir.sort()


train_files_name = [name for name in os.listdir(trainpath)]
test_files_name = [name for name in os.listdir(testpath)]
val_files_name =  [name for name in os.listdir(valpath)]


TRAINDIR = '/content/triple-mnist/train'
TESTDIR = '/content/triple-mnist/test'
VALDIR = '/content/triple-mnist/val'


In [9]:
# Empty list to store the training labels
label_y = []

def create_training_data():

    # Empty list to store each number image that is sent by each_digit function
    train_X = []

    # iterate over training directory
    for image_dir in train_dir:
        path = os.path.join(TRAINDIR,image_dir)  # path of directories

        # iterate over image directories in the training directory
        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                # extract andn convert to array
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

                # splitting digits by spaces between them (dividing image into 3 parts)
                img_array1 = img_array[:, :img_array.shape[1]//3]
                img_array2 = img_array[:, img_array.shape[1]//3: img_array.shape[1]//3 + img_array.shape[1]//3]
                img_array3 = img_array[:, img_array.shape[1]//3 + img_array.shape[1]//3 :]

                # append the splitted images of each digit
                train_X.append(each_digit(img_array1))
                train_X.append(each_digit(img_array2))
                train_X.append(each_digit(img_array3))

                # append the relative positioned number from the name of the image directory
                label_y.append(str(image_dir)[0])
                label_y.append(str(image_dir)[1])
                label_y.append(str(image_dir)[2])

            # in the interest in keeping the output clean...
            except Exception as e:
                pass
    return train_X

In [12]:
# Empty list to store the validation labels
val_y = []
def create_val_data():

    # Empty list to store each number image that is sent by each_digit function
    val_X = []

    # iterate over training directory
    for image_dir in val_dir:
        path = os.path.join(VALDIR,image_dir)

        # iterate over image directories in the training directory
        for img in tqdm(os.listdir(path)):
            try:
                # extract and convert to array
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

                # splitting digits by spaces between them (dividing image into 3 parts)
                img_array1 = img_array[:, :img_array.shape[1]//3]
                img_array2 = img_array[:, img_array.shape[1]//3: img_array.shape[1]//3 + img_array.shape[1]//3]
                img_array3 = img_array[:, img_array.shape[1]//3 + img_array.shape[1]//3 :]

                # append the splitted images of each digit
                val_X.append(each_digit(img_array1))
                val_X.append(each_digit(img_array2))
                val_X.append(each_digit(img_array3))


                # append the relative positioned number from the name of the image directory
                val_y.append(str(image_dir)[0])
                val_y.append(str(image_dir)[1])
                val_y.append(str(image_dir)[2])

            # in the interest in keeping the output clean...
            except Exception as e:
                pass
    return val_X

In [13]:
# Empty list to store the test labels
test_y = []

def create_test_data():

    # Empty list to store each number image that is sent by each_digit function
    test_X = []


    # iterate over training directory
    for image_dir in test_dir:
        path = os.path.join(TESTDIR,image_dir)

        # iterate over image directories in the training directory
        for img in tqdm(os.listdir(path)):
            try:
                # extract convert to array
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

                # splitting digits by spaces between them (dividing image into 3 parts)
                img_array1 = img_array[:, :img_array.shape[1]//3]
                img_array2 = img_array[:, img_array.shape[1]//3: img_array.shape[1]//3 + img_array.shape[1]//3]
                img_array3 = img_array[:, img_array.shape[1]//3 + img_array.shape[1]//3 :]

                # append the splitted images of each digit
                test_X.append(each_digit(img_array1))
                test_X.append(each_digit(img_array2))
                test_X.append(each_digit(img_array3))

                # append the relative positioned number from the name of the image directory
                test_y.append(str(image_dir)[0])
                test_y.append(str(image_dir)[1])
                test_y.append(str(image_dir)[2])

            # in the interest in keeping the output clean...
            except Exception as e:
                pass
    return test_X

In [14]:
def each_digit(image):

      ret, thresh = cv2.threshold(image.copy(), 0, 255, cv2.THRESH_BINARY_INV)

      contours, _  = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      for c in contours:
       x,y,w,h = cv2.boundingRect(c)

       # Creating a rectangle around the digit in the original image (for displaying the digits fetched via contours)
       cv2.rectangle(image, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=2)

       # Cropping out the digit from the image corresponding to the current contours in the for loop
       digit = thresh[y:y+h, x:x+w]

       # Resizing that digit to (28, 28)
       resized_digit = cv2.resize(digit, (28,28))

       # Adding the preprocessed digit to the list of preprocessed digits
       #preprocessed_digits.append(padded_digit)
       return resized_digit

In [21]:
td_array = np.array(create_training_data())
val_array = np.array(create_val_data())
test_array = np.array(create_test_data())

100%|██████████| 100/100 [00:00<00:00, 1510.32it/s]


In [24]:
td_array[0]

array([[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255,   0,   0,   0,
          0,   0,   0,   0, 255, 255, 255, 255,   0,   0,   0, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255,   0,   0,   0,   0, 255,
        255, 255, 255, 255, 255, 255, 255, 255,   0,   0,   0, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255,   0,   0,   0,   0, 255, 255, 255,
        255, 255, 255, 255, 255, 255,   0,   0,   0, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255,   0,   0,   0,   0, 255, 255, 255, 255,
        255, 255, 255,   0,   0,   0,   0,   0, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255],
       [255, 255, 255, 255, 255, 255, 25

In [29]:
class CNN(Sequential):
    def __init__(self,nfilters,sfilters):
        super().__init__()
        tf.random.set_seed(0)
        self.add(Conv2D(nfilters[0],kernel_size=(sfilters[0],sfilters[0]),padding='same',activation='relu',input_shape=(112,92,1)))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Conv2D(nfilters[1],kernel_size=(sfilters[1],sfilters[1]),padding='same',activation='relu'))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Conv2D(nfilters[2],kernel_size=(sfilters[2],sfilters[2]),padding='same',activation='relu'))
        self.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        self.add(Flatten())
        self.add(Dropout(0.3))
        self.add(Dense(512,activation='relu'))
        self.add(Dense(40,activation='softmax'))
        self.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])